# Processing Data

Import required packages and custom python functions for cleaning data

In [1]:
import json
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, text
from datetime import datetime

import data_cleaning



## 1. Read in Raw Data 
The data is found in the charts_song_data.json ad the unemployment.json

In [2]:
charts_df = pd.read_json("../data/raw/charts_song_data.json")
unemployment_df = pd.read_json("../data/raw/unemployment.json")

## 2. Clean and Process Data

Rename the columns in the Charts data frame for clarity, reformat the song and artist names for consistency. 

In [3]:
charts_df = charts_df.T.reset_index()
charts_df.columns = ["unique_id", "song", "artist", "track_id", "artist_id", "trending_year", "release_date", "genres"]
charts_df['song'] = charts_df['song'].str.lower().str.strip('"').str.strip("'").str.strip()
charts_df['artist'] = charts_df['artist'].str.lower().str.strip('"').str.strip("'").str.strip()

Split artists that collaborated on the same song into separate entities using custom function.

In [4]:
charts_df["artist"] = charts_df["artist"].apply(data_cleaning.split_artists)
charts_df = charts_df.explode("artist")

Select the year each song was released from the full date for easier analysis with unemployment statistics using custom function.

In [5]:
charts_df['release_year'] = charts_df['release_date'].apply(data_cleaning.clean_year)

Created broad genre categories (Rap/Hip-Hop, Pop, R&B, Rock, Jazz, Latin, Indie, Country) to draw connections between simmilar sub-genres. 

In [6]:
charts_df = charts_df.explode('genres').reset_index(drop=True)
charts_df['genre_category'] = charts_df['genres'].apply(data_cleaning.categorize_genre)

Clean and rename the unemployment statistical table, filter out for just the years I need (1970-2023; the range of release dates for the songs on the charts).

In [7]:
unemployment_df.columns = ["year", "unemployment_rate"]
unemployment_df['year'] = unemployment_df['year'].apply(data_cleaning.clean_year)
unemployment_df['year'] = pd.to_numeric(unemployment_df['year'], errors='coerce')
unemployment_df = unemployment_df[unemployment_df['year'] >= 1970]

## 3. Create SQL Data Base

Create a SQLite data base called topcharts with the following structure:
1. charts: track_id, trending year
2. songs: track_id, artist_id, track_title
3. artists: artist_id, artist_name
4. artist_genres: artist_id, genre
5. unemployment: year, unemployment_rate 

In [8]:
engine = create_engine('sqlite:///../data/topcharts.db', echo=False, isolation_level="AUTOCOMMIT")
with engine.connect() as conn:
    pass

Some primary keys had to be composite because there were non-unqiue columns. 
1. In the charts table, some songs were popular in more than one year. year was not unique because it applied to the 40 trending songs (per year). Therefore, the primary key, in order to be unique, had to be composite of trending year and track id
2. Similarly, for the artists table, if 2 artists collaborated on the same song, there was a unique artist id for their collaboration. I wanted the artist names serparated from the collaboration so that I could identify indivdual artists popularity (including collaborations), but this meant that artist_id was not unique as it was repeated for each artist in the collaboration (and artist name was also not unique because it would also repeat for multiple ids if they had collaborations). Therefore I had to create another composite key. 
3. For the artist_genre table, the artists were not unique because they were repeated for each genre corresponding to their name, and the genres were not unique because they were repeated for each artist they applied to. Again, I used a composite key to retain all the data.

In [9]:
create_statement_charts = """
    CREATE TABLE charts (
        track_id CHAR(25),
        trending_year DATETIME,
        PRIMARY KEY (track_id, trending_year)
        );
    """

create_statement_songs = """
    CREATE TABLE IF NOT EXISTS songs (
        track_id CHAR(25) PRIMARY KEY,
        track_title VARCHAR(100),
        artist_id CHAR(25),
        FOREIGN KEY (artist_id) REFERENCES songs(artist_id)
        );
    """

create_statement_artists = """
    CREATE TABLE IF NOT EXISTS artists (
    artist_id CHAR(22),
    artist_name VARCHAR(100),
    PRIMARY KEY (artist_id, artist_name)
);
"""

create_statement_artist_genres = """
    CREATE TABLE IF NOT EXISTS artist_genres (
        artist_id CHAR(25),
        genre VARCHAR(50),
        PRIMARY KEY (artist_id, genre),
        FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
        );
    """

create_statement_genres = """
    CREATE TABLE IF NOT EXISTS genres (
        sub_genre CHAR(25) PRIMARY KEY,
        genre VARCHAR(50),
        FOREIGN KEY (sub_genre) REFERENCES artist_genres(genre)
        );
    """

create_statement_unemployment = """ 
    CREATE TABLE IF NOT EXISTS unemployment (
        year DATETIME PRIMARY KEY,
        unemployment FLOAT
        );
    """

with engine.connect() as conn:
    conn.execute(text(create_statement_charts))
    conn.execute(text(create_statement_songs))
    conn.execute(text(create_statement_artists))
    conn.execute(text(create_statement_artist_genres))
    conn.execute(text(create_statement_unemployment))

In [10]:
charts_sql_df = charts_df[['track_id', 'trending_year']].drop_duplicates(subset=['track_id', 'trending_year'])
songs_sql_df = charts_df[['track_id', 'song', 'artist_id']].rename(columns={'song': 'track_title'}).drop_duplicates(subset='track_id', keep='last')
artists_sql_df = charts_df[['artist_id', 'artist']].rename(columns={'artist': 'artist_name'}).drop_duplicates()
artist_genres_df = charts_df[['artist_id', 'genres']].rename(columns={'genres': 'genre'}).drop_duplicates()
charts_sql_df = charts_df[['track_id', 'trending_year']].drop_duplicates(subset=['track_id', 'trending_year'])
genre_sql_df = charts_df[['genres', 'genre_category']].rename(columns={'genres': 'sub_genre', 'genre_category': 'genre'}).drop_duplicates()
unemployment_sql_df = unemployment_df.rename(columns={'unemployment_rate': 'unemployment'}).drop_duplicates()

with engine.connect() as conn:
    charts_sql_df.to_sql('charts', conn, if_exists='append', index=False)
    songs_sql_df.to_sql('songs', conn, if_exists='append', index=False)
    artists_sql_df.to_sql('artists', conn, if_exists='append', index=False)
    artist_genres_df.to_sql('artist_genres', conn, if_exists='append', index=False)
    genre_sql_df.to_sql('genres', conn, if_exists='append', index=False)
    unemployment_sql_df.to_sql('unemployment', conn, if_exists='append', index=False)
